<a href="https://colab.research.google.com/github/muhuuh/finetuning/blob/main/german_audio_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-l612cxq4/unsloth_5d75c67387e94c1087d3c5ad22d9dc43
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-l612cxq4/unsloth_5d75c67387e94c1087d3c5ad22d9dc43
  Resolved https://github.com/unslothai/unsloth.git to commit 4cb7229ac1c346e143524b6f9a6ad544259364d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 150.2 MB/s eta 0:00:00

In [6]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

In [9]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="LiquidAI/LFM2.5-1.2B-Instruct",  # native model for fine-tuning
    max_seq_length=2048,  # Matches model context; adjust based on your dataset
    dtype=None,  # Auto-detect (bfloat16 if available for better stability)
    load_in_4bit=True  # Enables 4-bit quantization for efficient fine-tuning
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Lfm2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [10]:
from datasets import load_dataset  # Import this for loading JSONL
import json

# Load the JSONL directly as a Hugging Face Dataset (handles large files efficiently)
ds = load_dataset("json", data_files="./sample_data/german_dictation_cleanup_dataset.jsonl")["train"]

def to_text(ex):
    resp = ex["response"]
    if not isinstance(resp, str):
        resp = json.dumps(resp, ensure_ascii=False)
    msgs = [
        {"role": "user", "content": ex["prompt"]},
        {"role": "assistant", "content": resp},
    ]
    return {
        "text": tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=False
        )
    }

# Apply mapping: Format into chat template and keep only "text" for SFTTrainer
dataset = ds.map(to_text, remove_columns=ds.column_names)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [15]:
print(dataset[3]["text"])

<|startoftext|><|im_start|>user
wir treffen uns um 14: 00 am Eingang und gehen dann zusammen rein.<|im_end|>
<|im_start|>assistant
Wir treffen uns um 14:00 am Eingang und gehen dann zusammen rein.<|im_end|>



In [19]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
import torch

# Apply LoRA adapters explicitly for quantized model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Low rank for efficiency
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory-saving during backprop
    random_state=3407,  # For reproducibility (like seed)
    max_seq_length=2048
)

# Now proceed with SFTTrainer as before
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=SFTConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=200,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs"
    )
)

trainer.train()

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


The model is already on multiple devices. Skipping the move to device specified in `args`.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,200 | Num Epochs = 3 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 884,736 of 1,171,225,344 (0.08% trained)


Step,Training Loss
1,4.310500
2,4.373600
3,4.471000
4,4.702800
5,4.686500
6,4.428300
7,4.598500
8,4.425100
9,4.361800
10,4.280300


TrainOutput(global_step=200, training_loss=1.930060277581215, metrics={'train_runtime': 234.6316, 'train_samples_per_second': 13.638, 'train_steps_per_second': 0.852, 'total_flos': 1151053565577216.0, 'train_loss': 1.930060277581215, 'epoch': 2.6666666666666665})

In [20]:
model.save_pretrained_gguf("german_audio_correction_gguf", tokenizer, quantization_method="q4_k_m", maximum_memory_usage = 0.7)

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `german_audio_correction_gguf`: 100%|██████████| 1/1 [00:43<00:00, 43.03s/it]


Successfully copied all 1 files from cache to `german_audio_correction_gguf`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:59<00:00, 59.23s/it]


Unsloth: Merge process complete. Saved to `/content/german_audio_correction_gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['LFM2.5-1.2B-Instruct.F

{'save_directory': 'german_audio_correction_gguf',
 'gguf_files': ['LFM2.5-1.2B-Instruct.Q4_K_M.gguf'],
 'modelfile_location': None,
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}